In [12]:
import torch
from torchvision import models
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
import json
import os
import time


# Load Pre-trained Model

In [2]:
resnet50 = models.resnet50(pretrained=True)


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


# Image Preprocessing

In [3]:
preprocess = transforms.Compose([
  transforms.Resize(256), 
  transforms.CenterCrop(224), 
  transforms.ToTensor(), 
  transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
  )])


# Load ImageNet label

In [4]:
# with open('./data/imagenet_classes.txt') as f:
#     labels = [ line.strip() for line in f.readlines() ]

# len(labels)


In [6]:
with open('./in_to_in9.json') as f:
    data = f.read()

in2in9_label = json.loads(data)


In [7]:
def getIn9Label(indexes):        
    return torch.tensor([in2in9_label[str(index.item())] for index in indexes])


# Load Testing Data

In [8]:
!wget -O mixed_same.tar.gz https://www.dropbox.com/s/f2525w5aqq67kk0/mixed_same.tar.gz?dl=0
!wget -O mixed_rand.tar.gz https://www.dropbox.com/s/cto15ceadgraur2/mixed_rand.tar.gz?dl=0
!wget -O only_bg_t.tar.gz https://www.dropbox.com/s/03lk878q73hyjpi/only_bg_t.tar.gz?dl=0
!wget -O original.tar.gz https://www.dropbox.com/s/0vv2qsc4ywb4z5v/original.tar.gz?dl=0


--2021-03-13 17:12:00--  https://www.dropbox.com/s/f2525w5aqq67kk0/mixed_same.tar.gz?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/f2525w5aqq67kk0/mixed_same.tar.gz [following]
--2021-03-13 17:12:00--  https://www.dropbox.com/s/raw/f2525w5aqq67kk0/mixed_same.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc277a78b3b12e4693978716563b.dl.dropboxusercontent.com/cd/0/inline/BKlwLUcRf2LjRNx8vhbV0Pv8-vSQYZwBbWcycGj3hfCbMOai7mppldqfobZaT2yq2GGXmNetkZqdpNFh31zoM7NK81e4p9dD_cvVvPj6J7HubdEqIeXqNq5yFyJmSskDpoT4IGWweATQTb7KN54gtW-U/file# [following]
--2021-03-13 17:12:01--  https://uc277a78b3b12e4693978716563b.dl.dropboxusercontent.com/cd/0/inline/BKlwLUcRf2LjRNx8vhbV0Pv8-vSQYZwBbWcycGj3hfCbMOai7mppldqfobZaT2yq2GGXmN

In [9]:
!tar -xvf 'mixed_same.tar.gz'
!tar -xvf 'mixed_rand.tar.gz'
!tar -xvf 'only_bg_t.tar.gz'
!tar -xvf 'original.tar.gz'

流式输出内容被截断，只能显示最后 5000 行内容。
original/train/00_dog/n02105641_1411.JPEG
original/train/00_dog/n02098105_195.JPEG
original/train/00_dog/n02105412_7465.JPEG
original/train/00_dog/n02088466_8904.JPEG
original/train/00_dog/n02091467_1213.JPEG
original/train/00_dog/n02086079_13106.JPEG
original/train/00_dog/n02088632_1583.JPEG
original/train/00_dog/n02090379_291.JPEG
original/train/00_dog/n02093991_1310.JPEG
original/train/00_dog/n02086240_3681.JPEG
original/train/00_dog/n02095570_4161.JPEG
original/train/00_dog/n02098105_2460.JPEG
original/train/00_dog/n02086240_4711.JPEG
original/train/00_dog/n02113186_12758.JPEG
original/train/00_dog/n02096294_484.JPEG
original/train/00_dog/n02090721_3527.JPEG
original/train/00_dog/n02099849_2315.JPEG
original/train/00_dog/n02091635_386.JPEG
original/train/00_dog/n02086646_1721.JPEG
original/train/00_dog/n02090622_8516.JPEG
original/train/00_dog/n02090622_2931.JPEG
original/train/00_dog/n02105412_3065.JPEG
original/train/00_dog/n02105412_5157.JPEG
original/

# Evaluation

In [13]:
class TestImage(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.images = []
        self.labels = []

        image_ext = ['.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif']
        for subdir in os.listdir(img_dir):
            subpath = os.path.join(img_dir, subdir)
            if os.path.isdir(subpath):
                label, _ = subdir.split('_')
                label = int(label)
                for f in os.listdir(subpath):
                    if any(f.lower().endswith(ext) for ext in image_ext):
                        self.images.append(f)
                        self.labels.append(label)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]

        img = imread(img_path)
        if self.transform:
            img_t = self.transform(img)
            batch_t = torch.unsqueeze(img_t, 0)

        return {
            "image": batch_t,
            'label': label
        }
    
    def __len__(self):
        return len(self.label)


In [14]:
def evaluation(test_dir):
    testset = ImageFolder(test_dir, transform=preprocess)
    testloader = DataLoader(testset, batch_size=64, shuffle=False)

    # Compute the model accuracy on the test set
    correct = 0
    total = 0

    for i, (inputs, target) in enumerate(testloader):
        inputs = inputs.cuda()
        out = resnet50(inputs)

        _, index = torch.max(out, 1)
        # percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        # labels[index], percentage[index].item()
        total += len(target)
        correct += (getIn9Label(index) == target).sum()

    print('Testing on', test_dir, 'Accuracy: %2.2f %%' % ((100.0 * correct) / total))

In [16]:
resnet50.eval()
resnet50 = resnet50.cuda()

eval_tests = ['./mixed_same/val', './mixed_rand/val', './only_bg_t/val', './original/val']

start = time.time()

for test_data in eval_tests:
    evaluation(test_data)

print(time.time() - start)

Testing on ./mixed_same/val Accuracy: 80.48 %
Testing on ./mixed_rand/val Accuracy: 69.77 %
Testing on ./only_bg_t/val Accuracy: 16.82 %
Testing on ./original/val Accuracy: 95.24 %
129.74640107154846
